# Advanced Regression Pipeline: XGBoost, and Random Forest Comparison

This notebook builds on the LightGBM pipeline to compare three regression algorithms using data from the local database. It includes data loading, preprocessing, model training, evaluation, and visualization.

In [1]:
import os
import json
import math
from datetime import datetime
from pathlib import Path
from typing import Dict, Tuple

## Environment
from dotenv import load_dotenv

## Core Scientific Stack
import numpy as np
import pandas as pd

## Visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Database
import psycopg2

## Machine Learning / Preprocessing (scikit-learn)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, Pipeline as SkPipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Gradient Boosting Libraries
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor

## Deep Learning / Tabular
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pytorch_tabnet.tab_model import TabNetRegressor

## Optimization & Persistence
import optuna
import joblib

# Load environment variables from .env file
load_dotenv(override=True)

# Setup model directory (handle notebook environment where __file__ is undefined)
try:
    PROJECT_ROOT = Path(__file__).resolve().parent
except NameError:
    # Fallback: assume notebook is inside src; go up one directory if so
    cwd = Path.cwd().resolve()
    if (cwd / 'supervised.ipynb').exists() or (cwd / 'unsupervised.ipynb').exists():
        PROJECT_ROOT = cwd
    else:
        for parent in cwd.parents:
            if (parent / 'requirements.txt').exists() or (parent / 'README.md').exists():
                PROJECT_ROOT = parent / 'src'
                break
        else:
            PROJECT_ROOT = cwd  # final fallback

MODEL_DIR = (PROJECT_ROOT / '..' / 'supervised').resolve()
MODEL_DIR.mkdir(parents=True, exist_ok=True)
print(f"Models will be saved to: {MODEL_DIR}")

def save_model(model, name: str, extra: dict | None = None):
    """Utility to persist models and optional metadata alongside them.
    Saves model as joblib plus a companion JSON with metadata/hyperparams."""
    timestamp = datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
    base_name = f"{name}_{timestamp}"
    model_path = MODEL_DIR / f"{base_name}.joblib"
    meta_path = MODEL_DIR / f"{base_name}.json"
    joblib.dump(model, model_path)
    meta = {'model_name': name, 'saved_utc': timestamp}
    if extra:
        meta.update(extra)
    with open(meta_path, 'w') as f:
        json.dump(meta, f, indent=2)
    print(f"Saved model -> {model_path.name}; metadata -> {meta_path.name}")

# Database connection parameters
db_params = {
    "host": os.getenv("LOCAL_HOST"),
    "user": os.getenv("LOCAL_USER"),
    "password": os.getenv("LOCAL_PW"),
    "port": os.getenv("LOCAL_PORT"),
    "dbname": os.getenv("LOCAL_DB")
}

# Display versions

print('LightGBM version:', lgb.__version__)
print('XGBoost version:', xgb.__version__)

Models will be saved to: D:\docs\MADS\696-Milestone 2\supervised
LightGBM version: 4.6.0
XGBoost version: 3.0.5


### Auto Load / Conditional Training
If a previously saved optimized model exists in `src/supervised`, the notebook will load the most recent artifact (by timestamp in filename) and skip retraining unless `FORCE_RETRAIN=True`.

In [10]:
# Auto-load previously saved optimized models (XGBoost / RandomForest / SVR / LinearRegression / Polynomial / MLP / TabNet)
from pathlib import Path as _Path
import json as _json

# Initialize placeholders if not already present
globals().setdefault('FORCE_RETRAIN', False)

# Only set to None if not defined to avoid clobbering models loaded earlier in session
if 'xgb_model' not in globals():
    xgb_model = None
if 'rf_model' not in globals():
    rf_model = None
if 'mlp_model' not in globals():
    mlp_model = None
if 'tabnet_model' not in globals():
    tabnet_model = None  # Will delay loading until wrapper class defined
if 'svr_model' not in globals():
    svr_model = None
if 'lr_model' not in globals():
    lr_model = None
if 'poly_model' not in globals():
    poly_model = None

MODEL_GLOB_PATTERNS = {
    'xgb_model': 'xgboost_opt_*.joblib',
    'rf_model': 'random_forest_opt_*.joblib', 
    'mlp_model': 'mlp_opt_*.joblib', 
    # 'tabnet_model': 'tabnet_opt_*.joblib',  # DEFERRED: load after wrapper defined
    'svr_model': 'svr_opt_*.joblib',
    'lr_model': 'linear_regression_opt_*.joblib',
    'poly_model': 'poly_reg_opt_*.joblib'
}

loaded_flags = {}
for var, pattern in MODEL_GLOB_PATTERNS.items():
    if globals().get(var) is not None:
        loaded_flags[var] = 'pre-existing'
        continue
    matches = sorted(MODEL_DIR.glob(pattern))
    if not matches:
        loaded_flags[var] = 'not found'
        continue
    latest = matches[-1]
    try:
        globals()[var] = joblib.load(latest)
        meta_file = latest.with_suffix('.json')
        if meta_file.exists():
            with open(meta_file) as f:
                globals()[f"{var}_meta"] = _json.load(f)
        loaded_flags[var] = f"loaded {latest.name}"
    except Exception as e:
        loaded_flags[var] = f"failed: {e}";
        globals()[var] = None

print("Auto-load status (TabNet deferred):")
for k,v in loaded_flags.items():
    print(f"  {k}: {v}")
print("FORCE_RETRAIN=", FORCE_RETRAIN)

Auto-load status (TabNet deferred):
  xgb_model: pre-existing
  rf_model: pre-existing
  mlp_model: pre-existing
  svr_model: pre-existing
  lr_model: pre-existing
  poly_model: pre-existing
FORCE_RETRAIN= False


In [3]:
# Connect to database and load data
try:
    conn = psycopg2.connect(**db_params)
    print("Database connection successful")
    sql_query = "SELECT * FROM dev.golden_table;"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    print("Golden data loaded into DataFrame:")
    print(df.info())
except Exception as e:
    print(f"An error occurred: {e}")

Database connection successful


C:\Users\Leo\AppData\Local\Temp\ipykernel_25204\3569683717.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Golden data loaded into DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23038 entries, 0 to 23037
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   school_name             23038 non-null  object 
 1   school_type             23038 non-null  object 
 2   teachers_fte            22550 non-null  float64
 3   enrollment              22863 non-null  float64
 4   grade_eight_enrollment  21613 non-null  float64
 5   math_counts             22507 non-null  float64
 6   math_high_pct           22507 non-null  float64
 7   math_low_pct            19960 non-null  float64
 8   read_counts             22386 non-null  float64
 9   read_high_pct           22386 non-null  float64
 10  read_low_pct            19907 non-null  float64
 11  pct_hhi_150k_200k       23038 non-null  float64
 12  pct_hhi_220k_plus       23038 non-null  float64
 13  avg_natwalkind          23038 non-null  float64
 14  tot

## 3. Data Splitting: Train, Validation, Test
Split the dataset into train, validation, and test sets, ensuring proper handling of the target variable.

In [4]:
# Define target and drop missing
TARGET = 'math_high_pct' if 'math_high_pct' in df.columns else 'target'
data = df.dropna().reset_index(drop=True)
drop_cols = ['hhi_150k_200k', 'hhi_220k_plus','total_10_14']
data = data.drop(columns=drop_cols)
data = data.set_index('school_name')

# Split features and target
feature_cols = [c for c in data.columns if c != TARGET and c != 'dup_rank']
X = data[feature_cols]
y = data[TARGET]

# Train/validation/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.40, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)
print(f'Train shape: {X_train.shape}, Validation shape: {X_valid.shape}, Test shape: {X_test.shape}')

Train shape: (11367, 16), Validation shape: (3789, 16), Test shape: (3789, 16)


## 4. Feature Engineering and Preprocessing Pipeline
Identify numeric and categorical features, set up StandardScaler and OneHotEncoder, and build a ColumnTransformer pipeline.

In [5]:
# Identify numeric and categorical features
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['category', 'object']).columns.tolist()

print('Numeric features:', numeric_features)
print('Categorical features:', categorical_features)

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# Fit preprocessor
preprocessor.fit(X_train)
X_train_enc = preprocessor.transform(X_train)
X_valid_enc = preprocessor.transform(X_valid)
X_test_enc = preprocessor.transform(X_test)

Numeric features: ['teachers_fte', 'enrollment', 'grade_eight_enrollment', 'math_counts', 'math_low_pct', 'read_counts', 'read_high_pct', 'read_low_pct', 'pct_hhi_150k_200k', 'pct_hhi_220k_plus', 'avg_natwalkind', 'pct_10_14', 'pct_female_10_14', 'total_pop', 'schools_in_zip']
Categorical features: ['school_type']


## Model Optimization function

In [6]:
# Reusable Optuna-based optimizer for cross-validated hyperparameter tuning
from typing import Callable, Dict, Tuple

def optimize_model_with_optuna(
                                model_name: str,
                                estimator_builder: Callable[[Dict], object],
                                param_space_fn: Callable[[optuna.trial.Trial], Dict],
                                X,
                                y,
                                scoring: str = 'neg_root_mean_squared_error',
                                cv: int = 3,
                                n_trials: int = 5,
                                direction: str = 'minimize',
                                random_state: int = 42,
                                n_jobs: int = -1,
                            ) -> Tuple[optuna.study.Study, Dict]:
    """Optimize a model's hyperparameters using Optuna and cross-validation.

    Args:
        model_name: Name used to label the Optuna study
        estimator_builder: Callable that receives a params dict and returns an unfitted estimator
        param_space_fn: Callable that maps an Optuna trial to a hyperparameter dictionary
        X, y: Training features and targets used for cross-validation
        scoring: scikit-learn scoring string guiding optimization
        cv: Number of cross-validation folds
        n_trials: Number of Optuna trials to run
        direction: 'minimize' or 'maximize' depending on the objective
        random_state: Seed for the Optuna sampler
        n_jobs: Parallelism for cross_val_score

    Returns:
        The completed Optuna study and the best hyperparameters discovered.
    """
    sampler = optuna.samplers.TPESampler(seed=random_state)
    study = optuna.create_study(study_name=f"{model_name}_opt", direction=direction, sampler=sampler)

    def objective(trial: optuna.trial.Trial) -> float:
        params = param_space_fn(trial)
        estimator = estimator_builder(params)
        scores = cross_val_score(estimator, X, y, cv=cv, scoring=scoring, n_jobs=n_jobs)
        mean_score = np.mean(scores)
        normalized_score = -mean_score if scoring.startswith('neg') else mean_score
        return normalized_score if direction == 'minimize' else -normalized_score

    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    return study, study.best_params

## Model metrics

## LINEAR MODELS


In [7]:
# Linear & Kernel-based Models: SVR + Polynomial Regression (with Optuna tuning + CV)
def build_svr_estimator(params: Dict) -> SVR:
    base = {'kernel': params.get('kernel', 'rbf')}
    # Map params safely
    for k in ['C','epsilon','gamma','degree']:
        if k in params:
            base[k] = params[k]
    return SVR(**base)

def svr_param_space(trial: optuna.trial.Trial) -> Dict:
    kernel = trial.suggest_categorical('kernel', ['rbf','poly','sigmoid'])
    params = {
        'kernel': kernel,
        'C': trial.suggest_float('C', 1e-1, 1e3, log=True),
        'epsilon': trial.suggest_float('epsilon', 1e-3, 0.5, log=True)
    }
    if kernel in ['rbf','sigmoid']:
        params['gamma'] = trial.suggest_float('gamma', 1e-4, 1, log=True)
    if kernel == 'poly':
        params['degree'] = trial.suggest_int('degree', 2, 5)
        params['gamma'] = trial.suggest_float('gamma', 1e-4, 1, log=True)
    return params

def build_poly_estimator(params: Dict):
    degree = params.get('degree', 2)
    include_bias = params.get('include_bias', False)
    interaction_only = params.get('interaction_only', False)
    return SkPipeline([
        ('poly', PolynomialFeatures(degree=degree, include_bias=include_bias, interaction_only=interaction_only)),
        ('lr', LinearRegression())
    ])

def poly_param_space(trial: optuna.trial.Trial) -> Dict:
    return {
        'degree': trial.suggest_int('degree', 2, 5),
        'include_bias': False,
        'interaction_only': trial.suggest_categorical('interaction_only', [False, True])
    }

# Run / reuse SVR optimization
if svr_model is None or FORCE_RETRAIN:
    print('[SVR] Starting Optuna optimization...')
    svr_study, svr_best_params = optimize_model_with_optuna(
        model_name='SVR',
        estimator_builder=build_svr_estimator,
        param_space_fn=svr_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=5,
        direction='minimize',
        random_state=42,
        n_jobs=-1,
    )
    print('Best SVR params:', svr_best_params)
    svr_model = build_svr_estimator(svr_best_params)
    svr_model.fit(X_train_enc, y_train)
    svr_valid_pred = svr_model.predict(X_valid_enc)
    svr_test_pred = svr_model.predict(X_test_enc)
    save_model(svr_model, 'svr_opt', {'best_params': svr_best_params})
else:
    print('[SVR] Using preloaded optimized model; generating predictions.')
    svr_valid_pred = svr_model.predict(X_valid_enc)
    svr_test_pred = svr_model.predict(X_test_enc)

# Baseline Linear Regression (also optionally re-optimized via polynomial)
if lr_model is None and poly_model is None:
    # Keep a simple baseline linear regression for reference
    lr_model = LinearRegression()
    lr_model.fit(X_train_enc, y_train)
    save_model(lr_model, 'linear_regression_opt', {'params': lr_model.get_params(), 'baseline': True})
    lr_valid_pred = lr_model.predict(X_valid_enc)
    lr_test_pred = lr_model.predict(X_test_enc)
elif lr_model is not None:
    lr_valid_pred = lr_model.predict(X_valid_enc)
    lr_test_pred = lr_model.predict(X_test_enc)

# Polynomial Regression optimization
if poly_model is None or FORCE_RETRAIN:
    print('[PolynomialRegression] Starting Optuna optimization...')
    poly_study, poly_best_params = optimize_model_with_optuna(
        model_name='PolynomialRegression',
        estimator_builder=build_poly_estimator,
        param_space_fn=poly_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=5,
        direction='minimize',
        random_state=42,
        n_jobs=-1,
    )
    print('Best Polynomial Regression params:', poly_best_params)
    poly_model = build_poly_estimator(poly_best_params)
    poly_model.fit(X_train_enc, y_train)
    poly_valid_pred = poly_model.predict(X_valid_enc)
    poly_test_pred = poly_model.predict(X_test_enc)
    save_model(poly_model, 'poly_reg_opt', {'best_params': poly_best_params})
else:
    print('[PolynomialRegression] Using preloaded optimized model; generating predictions.')
    poly_valid_pred = poly_model.predict(X_valid_enc)
    poly_test_pred = poly_model.predict(X_test_enc)

[SVR] Using preloaded optimized model; generating predictions.


[I 2025-10-06 10:15:39,164] A new study created in memory with name: PolynomialRegression_opt


[PolynomialRegression] Starting Optuna optimization...


  0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-10-06 10:15:46,883] Trial 0 finished with value: 4937230285.657545 and parameters: {'degree': 3, 'interaction_only': False}. Best is trial 0 with value: 4937230285.657545.
[I 2025-10-06 10:23:42,664] Trial 1 finished with value: 14761702710.802286 and parameters: {'degree': 4, 'interaction_only': False}. Best is trial 0 with value: 4937230285.657545.
[I 2025-10-06 10:23:42,664] Trial 1 finished with value: 14761702710.802286 and parameters: {'degree': 4, 'interaction_only': False}. Best is trial 0 with value: 4937230285.657545.
[I 2025-10-06 10:23:43,103] Trial 2 finished with value: 377312545695.4566 and parameters: {'degree': 2, 'interaction_only': False}. Best is trial 0 with value: 4937230285.657545.
[I 2025-10-06 10:23:43,103] Trial 2 finished with value: 377312545695.4566 and parameters: {'degree': 2, 'interaction_only': False}. Best is trial 0 with value: 4937230285.657545.
[I 2025-10-06 10:26:41,201] Trial 3 finished with value: 281993117177.21075 and parameters: {'degr

# NEURAL NETWORKS


## MLP

In [8]:
# Torch-based MLP (CUDA-enabled) + TabNet parameter spaces and optimization hooks

def _ensure_dense_np(X):
    # ColumnTransformer output may be sparse
    if hasattr(X, 'toarray'):
        return X.toarray()
    return X

def _set_torch_seed(seed: int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

class TorchMLPRegressor(BaseEstimator, RegressorMixin):
    """A lightweight PyTorch MLP for regression with sklearn API.

    Supports:
      - hidden_layer_sizes: tuple of ints
      - activation: 'relu' | 'tanh'
      - learning_rate_init: float (alias learning_rate)
      - alpha: L2 weight decay
      - batch_size, max_iter (epochs)
      - early_stopping with validation split + patience (n_iter_no_change)
      - CUDA automatically if available (override via device)
    """
    def __init__(
        self,
        hidden_layer_sizes: Tuple[int, ...] = (128, 64),
        activation: str = 'relu',
        learning_rate_init: float = 1e-3,
        alpha: float = 0.0,
        batch_size: int = 128,
        max_iter: int = 100,
        early_stopping: bool = True,
        validation_fraction: float = 0.1,
        n_iter_no_change: int = 15,
        random_state: int = 42,
        verbose: bool = False,
        device: str | None = None,
    ):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.learning_rate_init = learning_rate_init
        self.alpha = alpha
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.early_stopping = early_stopping
        self.validation_fraction = validation_fraction
        self.n_iter_no_change = n_iter_no_change
        self.random_state = random_state
        self.verbose = verbose
        self.device = device  # 'cuda' | 'cpu' | None

    def get_params(self, deep=True):  # needed for sklearn clone
        return {
            'hidden_layer_sizes': self.hidden_layer_sizes,
            'activation': self.activation,
            'learning_rate_init': self.learning_rate_init,
            'alpha': self.alpha,
            'batch_size': self.batch_size,
            'max_iter': self.max_iter,
            'early_stopping': self.early_stopping,
            'validation_fraction': self.validation_fraction,
            'n_iter_no_change': self.n_iter_no_change,
            'random_state': self.random_state,
            'verbose': self.verbose,
            'device': self.device,
        }

    def set_params(self, **params):  # compatibility
        for k, v in params.items():
            setattr(self, k, v)
        return self

    def _build_network(self, in_features: int):
        layers: list[nn.Module] = []
        prev = in_features
        act_map = {'relu': nn.ReLU, 'tanh': nn.Tanh}
        Act = act_map.get(self.activation.lower(), nn.ReLU)
        for h in self.hidden_layer_sizes:
            layers.append(nn.Linear(prev, h))
            layers.append(Act())
            prev = h
        layers.append(nn.Linear(prev, 1))
        return nn.Sequential(*layers)

    def fit(self, X, y):
        _set_torch_seed(self.random_state)
        X_np = _ensure_dense_np(X).astype('float32')
        y_np = (y.values if hasattr(y, 'values') else y).astype('float32').reshape(-1, 1)
        n_samples = X_np.shape[0]
        # Validation split
        if self.early_stopping and 0 < self.validation_fraction < 0.5 and n_samples > 20:
            val_size = max(1, int(n_samples * self.validation_fraction))
            idx = np.arange(n_samples)
            rng = np.random.default_rng(self.random_state)
            rng.shuffle(idx)
            val_idx = idx[:val_size]
            train_idx = idx[val_size:]
            X_train, y_train = X_np[train_idx], y_np[train_idx]
            X_val, y_val = X_np[val_idx], y_np[val_idx]
        else:
            X_train, y_train = X_np, y_np
            X_val, y_val = None, None
        device = self.device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self._device_ = device
        # Explicit device print for visibility
        print(f"[TorchMLP] Training on device: {device}")
        self.model_ = self._build_network(X_np.shape[1]).to(device)
        self.optimizer_ = torch.optim.Adam(self.model_.parameters(), lr=self.learning_rate_init, weight_decay=self.alpha)
        self.criterion_ = nn.MSELoss()
        train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
        train_loader = DataLoader(train_ds, batch_size=self.batch_size, shuffle=True)
        if X_val is not None:
            val_tensor = torch.from_numpy(X_val).to(device)
            y_val_tensor = torch.from_numpy(y_val).to(device)
        best_val = math.inf
        best_state = None
        rounds_no_improve = 0
        for epoch in range(1, self.max_iter + 1):
            self.model_.train()
            epoch_loss = 0.0
            for xb, yb in train_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                self.optimizer_.zero_grad()
                pred = self.model_(xb)
                loss = self.criterion_(pred, yb)
                loss.backward()
                self.optimizer_.step()
                epoch_loss += loss.item() * xb.size(0)
            epoch_loss /= len(train_loader.dataset)
            if X_val is not None:
                self.model_.eval()
                with torch.no_grad():
                    val_pred = self.model_(val_tensor)
                    val_loss = self.criterion_(val_pred, y_val_tensor).item()
                improved = val_loss + 1e-9 < best_val
                if improved:
                    best_val = val_loss
                    best_state = {k: v.cpu().clone() for k, v in self.model_.state_dict().items()}
                    rounds_no_improve = 0
                else:
                    rounds_no_improve += 1
                if self.verbose and (epoch % 10 == 0 or improved):
                    print(f"Epoch {epoch:03d} train_loss={epoch_loss:.4f} val_loss={val_loss:.4f} best_val={best_val:.4f}")
                if self.early_stopping and rounds_no_improve >= self.n_iter_no_change:
                    if self.verbose:
                        print(f"Early stopping at epoch {epoch}")
                    break
            else:
                if self.verbose and epoch % 10 == 0:
                    print(f"Epoch {epoch:03d} train_loss={epoch_loss:.4f}")
        # Load best validation weights
        if best_state is not None:
            self.model_.load_state_dict(best_state)
        self.n_features_in_ = X_np.shape[1]
        return self

    def predict(self, X):
        check_is_fitted(self, ['model_'])
        X_np = _ensure_dense_np(X).astype('float32')
        device = self._device_
        self.model_.eval()
        with torch.no_grad():
            preds = self.model_(torch.from_numpy(X_np).to(device))
        return preds.cpu().numpy().ravel()

def build_mlp_estimator(params: Dict) -> TorchMLPRegressor:
    """Return a TorchMLPRegressor; map sklearn-style names used in param space.
    Removes Optuna helper keys hl1/hl2 before constructing."""
    params = params.copy()
    params.pop('hl1', None)
    params.pop('hl2', None)
    return TorchMLPRegressor(**params)

def mlp_param_space(trial: optuna.trial.Trial) -> Dict:
    return {
        'hidden_layer_sizes': tuple(sorted([
            trial.suggest_int('hl1', 64, 256, step=32),
            trial.suggest_int('hl2', 32, 192, step=32)
        ], reverse=True)),
        'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True),
        'alpha': trial.suggest_float('alpha', 1e-6, 1e-2, log=True),  # weight decay
        'batch_size': trial.suggest_categorical('batch_size', [64, 128, 256]),
        'max_iter': trial.suggest_int('max_iter', 150, 600, step=75),
        'n_iter_no_change': trial.suggest_int('n_iter_no_change', 5, 25, step=5),
        'early_stopping': True,
        'validation_fraction': 0.15,
        'activation': trial.suggest_categorical('activation', ['relu', 'tanh']),
        'random_state': 42,
        'verbose': False,
    }

# Run / reuse MLP optimization
if mlp_model is None or FORCE_RETRAIN:
    print("[TorchMLP] No preloaded model (or FORCE_RETRAIN=True). Starting Optuna optimization on", ('CUDA' if torch.cuda.is_available() else 'CPU'))
    mlp_study, mlp_best_params = optimize_model_with_optuna(
        model_name='TorchMLPRegressor',
        estimator_builder=build_mlp_estimator,
        param_space_fn=mlp_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=5,
        direction='minimize',
        random_state=42,
        n_jobs=-1,
    )
    print('Best TorchMLP params:', mlp_best_params)
    mlp_model = build_mlp_estimator(mlp_best_params)
    mlp_model.fit(X_train_enc, y_train)
    mlp_valid_pred = mlp_model.predict(X_valid_enc)
    mlp_test_pred = mlp_model.predict(X_test_enc)
    save_model(mlp_model, 'mlp_opt', {'best_params': mlp_best_params, 'framework': 'torch'})
else:
    # Refitting even when a model was preloaded to align with current dataset / preprocessing state
    print("[TorchMLP] Preloaded model found; refitting on current data to refresh weights and predictions.")
    # Attempt to reuse stored best params if they exist on the object; else use its own get_params()
    try:
        existing_params = {k: v for k, v in mlp_model.get_params().items() if k in TorchMLPRegressor().get_params()}
    except Exception:
        existing_params = {}
    mlp_model = build_mlp_estimator(existing_params) if existing_params else mlp_model
    mlp_model.fit(X_train_enc, y_train)
    mlp_valid_pred = mlp_model.predict(X_valid_enc)
    mlp_test_pred = mlp_model.predict(X_test_enc)
    # Optionally save refit (comment out if not desired)
    save_model(mlp_model, 'mlp_opt_refit', {'refit': True, 'framework': 'torch'})

[I 2025-10-06 11:51:49,263] A new study created in memory with name: TorchMLPRegressor_opt


[TorchMLP] No preloaded model (or FORCE_RETRAIN=True). Starting Optuna optimization on CPU


  0%|          | 0/5 [00:00<?, ?it/s]

[I 2025-10-06 11:52:14,379] Trial 0 finished with value: 12.283570654990045 and parameters: {'hl1': 128, 'hl2': 192, 'learning_rate_init': 0.0029106359131330704, 'alpha': 0.0002481040974867811, 'batch_size': 64, 'max_iter': 600, 'n_iter_no_change': 20, 'activation': 'relu'}. Best is trial 0 with value: 12.283570654990045.
[I 2025-10-06 11:52:46,646] Trial 1 finished with value: 12.345241395345903 and parameters: {'hl1': 256, 'hl2': 160, 'learning_rate_init': 0.00026587543983272726, 'alpha': 5.337032762603957e-06, 'batch_size': 256, 'max_iter': 375, 'n_iter_no_change': 10, 'activation': 'relu'}. Best is trial 0 with value: 12.283570654990045.
[I 2025-10-06 11:52:46,646] Trial 1 finished with value: 12.345241395345903 and parameters: {'hl1': 256, 'hl2': 160, 'learning_rate_init': 0.00026587543983272726, 'alpha': 5.337032762603957e-06, 'batch_size': 256, 'max_iter': 375, 'n_iter_no_change': 10, 'activation': 'relu'}. Best is trial 0 with value: 12.283570654990045.
[I 2025-10-06 11:53:00,9

## TABNET

In [11]:
def _to_float32_dense(X):
    return X.toarray().astype(np.float32) if hasattr(X, 'toarray') else np.asarray(X, dtype=np.float32)

# Speed/behavior flags
globals().setdefault('FAST_TABNET', True)  # Toggle this to False for full/slow training

class TabNetRegressorWrapper(BaseEstimator, RegressorMixin):
    """A scikit-learn compatible wrapper for TabNetRegressor with fast-mode support.

    Features:
      - Converts sparse inputs to dense float32
      - Reshapes y to (n,1) as required by TabNet
      - Internal validation split (validation_fraction) enabling effective early stopping
      - Fast mode reducing epochs, patience, and search space for quicker experimentation
    """
    def __init__(self,
                 max_epochs=200,
                 patience=50,
                 batch_size=1024,
                 virtual_batch_size=128,
                 eval_metric='rmse',
                 validation_fraction: float = 0.15,
                 seed=42,
                 verbose=0,
                 fast: bool | None = None,
                 **tabnet_params):
        self.max_epochs = max_epochs
        self.patience = patience
        self.batch_size = batch_size
        self.virtual_batch_size = virtual_batch_size
        self.eval_metric = eval_metric if isinstance(eval_metric, (list, tuple)) else [eval_metric]
        self.validation_fraction = validation_fraction
        self.seed = seed
        self.verbose = verbose
        self.fast = FAST_TABNET if fast is None else fast
        self.tabnet_params = tabnet_params
        self.model_ = None

    def _to_dense32(self, X):
        return _to_float32_dense(X)

    def fit(self, X, y):
        Xd = self._to_dense32(X)
        y_arr = np.asarray(y)
        if y_arr.ndim == 1:
            y_arr = y_arr.reshape(-1, 1)
        n = Xd.shape[0]
        # Internal validation split for genuine early stopping
        if 0 < self.validation_fraction < 0.5 and n > 30:
            val_size = max(1, int(n * self.validation_fraction))
            rng = np.random.default_rng(self.seed)
            idx = rng.permutation(n)
            val_idx = idx[:val_size]
            train_idx = idx[val_size:]
            X_train, y_train = Xd[train_idx], y_arr[train_idx]
            X_val, y_val = Xd[val_idx], y_arr[val_idx]
            eval_set = [(X_val, y_val)]
        else:
            X_train, y_train = Xd, y_arr
            eval_set = [(Xd, y_arr)]
        # Adjust hyperparameters in fast mode (applies only at fit time)
        max_epochs = min(self.max_epochs, 120 if self.fast else self.max_epochs)
        patience = min(self.patience, 25 if self.fast else self.patience)
        if self.fast and self.verbose:
            print(f"[TabNetWrapper] FAST mode active: max_epochs={max_epochs}, patience={patience}")
        self.model_ = TabNetRegressor(seed=self.seed, verbose=self.verbose, **self.tabnet_params)
        self.model_.fit(
            X_train,
            y_train,
            eval_set=eval_set,
            eval_metric=self.eval_metric,
            patience=patience,
            max_epochs=max_epochs,
            batch_size=self.batch_size,
            virtual_batch_size=self.virtual_batch_size,
        )
        return self

    def predict(self, X):
        check_is_fitted(self, 'model_')
        Xd = self._to_dense32(X)
        preds = self.model_.predict(Xd)
        return preds.ravel()


def build_tabnet_estimator(params: Dict) -> TabNetRegressorWrapper:
    """Return a TabNetRegressorWrapper with defaults merged with tuned params."""
    defaults = dict(
        n_d=16, n_a=16, n_steps=5, gamma=1.3,
        lambda_sparse=1e-4,
        optimizer_params={'lr': 2e-2},
    )
    defaults.update(params)
    training_keys = ['max_epochs', 'patience', 'batch_size', 'virtual_batch_size', 'validation_fraction', 'fast']
    training_overrides = {k: defaults.pop(k) for k in list(defaults.keys()) if k in training_keys}
    return TabNetRegressorWrapper(**training_overrides, **defaults)


def tabnet_param_space(trial: optuna.trial.Trial) -> Dict:
    """Reduced/narrower search space for faster optimization when FAST_TABNET is True."""
    fast = FAST_TABNET
    return {
        'n_d': trial.suggest_categorical('n_d', [8, 16, 24] if fast else [8, 16, 24, 32]),
        'n_a': trial.suggest_categorical('n_a', [8, 16, 24] if fast else [8, 16, 24, 32]),
        'n_steps': trial.suggest_int('n_steps', 3, 6 if fast else 8),
        'gamma': trial.suggest_float('gamma', 1.0, 1.8 if fast else 2.0),
        'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-6, 5e-4 if fast else 1e-3, log=True),
        'optimizer_params': {'lr': trial.suggest_float('lr', 1e-3, 2e-2 if fast else 5e-2, log=True)},
        'max_epochs': 100 if fast else 200,
        'patience': 20 if fast else 50,
        'batch_size': 1024,
        'virtual_batch_size': 128,
        'validation_fraction': 0.15,
        'fast': fast,
    }

# Late-load TabNet model now that wrapper class exists (if not already loaded)
if tabnet_model is None and not FORCE_RETRAIN:
    tabnet_matches = sorted(MODEL_DIR.glob('tabnet_opt_*.joblib'))
    if tabnet_matches:
        latest_tabnet = tabnet_matches[-1]
        try:
            tabnet_model = joblib.load(latest_tabnet)
            meta_file = latest_tabnet.with_suffix('.json')
            if meta_file.exists():
                with open(meta_file) as f:
                    tabnet_model_meta = json.load(f)
            print(f"[TabNet] Late-loaded saved model: {latest_tabnet.name}")
        except Exception as e:
            print(f"[TabNet] Late-load failed: {e}")

# Run TabNet optimization (wrapper handles dense conversion & y reshape)
if tabnet_model is None or FORCE_RETRAIN:
    print(f"[TabNet] Starting Optuna optimization (FAST_TABNET={FAST_TABNET})")
    tabnet_study, tabnet_best_params = optimize_model_with_optuna(
        model_name='TabNetRegressor',
        estimator_builder=build_tabnet_estimator,
        param_space_fn=tabnet_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=5 if FAST_TABNET else 15,
        direction='minimize',
        random_state=42,
        n_jobs=-1,
    )
    print('Best TabNetRegressor params:', tabnet_best_params)
    tabnet_model = build_tabnet_estimator(tabnet_best_params)
    tabnet_model.fit(X_train_enc, y_train)
    tabnet_valid_pred = tabnet_model.predict(X_valid_enc)
    tabnet_test_pred = tabnet_model.predict(X_test_enc)
    save_model(tabnet_model, 'tabnet_opt', {'best_params': tabnet_best_params, 'fast_mode': FAST_TABNET})
else:
    # Refit even when preloaded to ensure model aligns with current transformed features
    print(f"[TabNet] Using loaded model; refitting on current data (FAST_TABNET={FAST_TABNET}).")
    reuse_params = None
    try:
        if 'tabnet_model_meta' in globals() and isinstance(tabnet_model_meta, dict):
            reuse_params = tabnet_model_meta.get('best_params')
    except Exception:
        reuse_params = None
    if reuse_params is None:
        reuse_params = {}
    reuse_params['fast'] = FAST_TABNET
    tabnet_model = build_tabnet_estimator(reuse_params)
    tabnet_model.fit(X_train_enc, y_train)
    tabnet_valid_pred = tabnet_model.predict(X_valid_enc)
    tabnet_test_pred = tabnet_model.predict(X_test_enc)
    save_model(tabnet_model, 'tabnet_opt_refit', {'refit': True, 'best_params': reuse_params, 'fast_mode': FAST_TABNET})

[TabNet] Late-loaded saved model: tabnet_opt_refit_20251005T212433Z.joblib
[TabNet] Using loaded model; refitting on current data (FAST_TABNET=True).
Stop training because you reached max_epochs = 120 with best_epoch = 108 and best_val_0_rmse = 10.15413
Stop training because you reached max_epochs = 120 with best_epoch = 108 and best_val_0_rmse = 10.15413


c:\Users\Leo\miniconda3\envs\milestone2\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Saved model -> tabnet_opt_refit_20251006T155735Z.joblib; metadata -> tabnet_opt_refit_20251006T155735Z.json


# 5. Tree-based ensemble models

## XGBOOST

In [12]:
# Hyperparameter optimization and training for ensemble models
def build_xgb_estimator(params: Dict) -> XGBRegressor:
    base_params = {
        'random_state': 42,
        'device': 'cuda',
        'verbosity': 0,
        'tree_method': 'gpu_hist'
    }
    base_params.update(params)
    return XGBRegressor(**base_params)


def xgb_param_space(trial: optuna.trial.Trial) -> Dict:
    return {
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1e-1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1e-1, log=True)
    }

if xgb_model is None or FORCE_RETRAIN:
    print("[XGBoost] No preloaded model (or FORCE_RETRAIN=True). Starting Optuna optimization...")
    xgb_study, xgb_best_params = optimize_model_with_optuna(
        model_name='XGBoost',
        estimator_builder=build_xgb_estimator,
        param_space_fn=xgb_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=15,
        direction='minimize',
        random_state=42,
        n_jobs=-1
    )
    print('Best XGBoost params:', xgb_best_params)
    # Fit model with optimized hyperparameters
    xgb_model = build_xgb_estimator(xgb_best_params)
    xgb_model.fit(X_train_enc, y_train)
    xgb_valid_pred = xgb_model.predict(X_valid_enc)
    xgb_test_pred = xgb_model.predict(X_test_enc)
    save_model(xgb_model, 'xgboost_opt', {'best_params': xgb_best_params})
else:
    # Refit even when preloaded to ensure alignment with current data & preprocessing
    print('[XGBoost] Preloaded model found; refitting on current data.')
    # Try to pull previously stored best params from metadata if available
    reuse_params = None
    try:
        if 'xgb_model_meta' in globals() and isinstance(xgb_model_meta, dict):
            reuse_params = xgb_model_meta.get('best_params')
    except Exception:
        reuse_params = None
    if reuse_params is None:
        # Fall back to current model's parameters (filter to search space + core)
        try:
            current = xgb_model.get_params()
            reuse_keys = {'n_estimators','max_depth','learning_rate','subsample','colsample_bytree','reg_alpha','reg_lambda'}
            reuse_params = {k: v for k, v in current.items() if k in reuse_keys}
        except Exception:
            reuse_params = {}
    # Rebuild a fresh estimator to avoid any internal state carry-over
    xgb_model = build_xgb_estimator(reuse_params)
    xgb_model.fit(X_train_enc, y_train)
    xgb_valid_pred = xgb_model.predict(X_valid_enc)
    xgb_test_pred = xgb_model.predict(X_test_enc)
    # Save refit artifact
    save_model(xgb_model, 'xgboost_opt_refit', {'refit': True, 'best_params': reuse_params})

[XGBoost] Preloaded model found; refitting on current data.
Saved model -> xgboost_opt_refit_20251006T160058Z.joblib; metadata -> xgboost_opt_refit_20251006T160058Z.json
Saved model -> xgboost_opt_refit_20251006T160058Z.joblib; metadata -> xgboost_opt_refit_20251006T160058Z.json


# RANDOM FOREST

In [13]:
def build_rf_estimator(params: Dict) -> RandomForestRegressor:
    base_params = {
        'random_state': 42,
        'n_jobs': -1
    }
    base_params.update(params)
    return RandomForestRegressor(**base_params)


def rf_param_space(trial: optuna.trial.Trial) -> Dict:
    return {
        'n_estimators': trial.suggest_int('n_estimators', 200, 800),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_float('max_features', 0.4, 1.0)
    }

if rf_model is None or FORCE_RETRAIN:
    print("[RandomForest] No preloaded model (or FORCE_RETRAIN=True). Starting Optuna optimization...")
    rf_study, rf_best_params = optimize_model_with_optuna(
        model_name='RandomForest',
        estimator_builder=build_rf_estimator,
        param_space_fn=rf_param_space,
        X=X_train_enc,
        y=y_train,
        scoring='neg_root_mean_squared_error',
        cv=3,
        n_trials=25,
        direction='minimize',
        random_state=42,
        n_jobs=-1,
    )
    print('Best Random Forest params:', rf_best_params)
    rf_model = build_rf_estimator(rf_best_params)
    rf_model.fit(X_train_enc, y_train)
    rf_valid_pred = rf_model.predict(X_valid_enc)
    rf_test_pred = rf_model.predict(X_test_enc)
    save_model(rf_model, 'random_forest_opt', {'best_params': rf_best_params})
else:
    print("[RandomForest] Using preloaded optimized model. Skipping training.")
    rf_valid_pred = rf_model.predict(X_valid_enc)
    rf_test_pred = rf_model.predict(X_test_enc)


[RandomForest] Using preloaded optimized model. Skipping training.


## 6. Compare Model Performance
Evaluate predictions from each model using RMSE, MAE, and R² metrics.

In [14]:
# Helper functions for metrics

def adjusted_r2_score(r2, n, p):
    return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

def regression_metrics(y_true, y_pred, X=None):
    try:
        # Try using root_mean_squared_error if available (sklearn >= 1.4)
        from sklearn.metrics import root_mean_squared_error
        rmse = root_mean_squared_error(y_true, y_pred)
    except ImportError:
        # Fall back to mean_squared_error with squared=False for older versions
        rmse = mean_squared_error(y_true, y_pred, squared=False)
    
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    metrics = {'RMSE': rmse, 'MAE': mae, 'R2': r2}
    if X is not None:
        n = X.shape[0]
        p = X.shape[1]
        adj_r2 = adjusted_r2_score(r2, n, p)
        metrics['Adj_R2'] = adj_r2
    return metrics


In [ ]:
# Comprehensive Model Performance Comparison & Best Model Selection (All Methods)
import json as _json_summary
from copy import deepcopy

# Utility to (re)compute metrics if predictions exist and metrics missing
def _ensure_metrics(prefix: str, display_name: str, Xv, Xt):
    g = globals()
    valid_pred_var = f'{prefix}_valid_pred'
    test_pred_var = f'{prefix}_test_pred'
    valid_metrics_var = f'{prefix}_metrics'  # normalized name for validation metrics
    test_metrics_var = f'{prefix}_test_metrics'
    # Accept older naming conventions (e.g., rf_metrics already used)
    if valid_pred_var in g:
        if valid_metrics_var not in g or g.get(valid_metrics_var) is None:
            try:
                g[valid_metrics_var] = regression_metrics(y_valid, g[valid_pred_var], Xv)
            except Exception as e:
                print(f'[WARN] Could not compute validation metrics for {display_name}: {e}')
                g[valid_metrics_var] = None
    if test_pred_var in g:
        if test_metrics_var not in g or g.get(test_metrics_var) is None:
            try:
                g[test_metrics_var] = regression_metrics(y_test, g[test_pred_var], Xt)
            except Exception as e:
                print(f'[WARN] Could not compute test metrics for {display_name}: {e}')
                g[test_metrics_var] = None
    return (g.get(valid_metrics_var), g.get(test_metrics_var))

# Recompute core ensemble metrics if missing (guards for partial executions)
# This path is now superseded by _ensure_metrics but kept for clarity
if 'xgb_metrics' not in globals() and 'xgb_valid_pred' in globals():
    xgb_metrics = regression_metrics(y_valid, xgb_valid_pred, X_valid_enc)
if 'rf_metrics' not in globals() and 'rf_valid_pred' in globals():
    rf_metrics = regression_metrics(y_valid, rf_valid_pred, X_valid_enc)
if 'xgb_test_metrics' not in globals() and 'xgb_test_pred' in globals():
    xgb_test_metrics = regression_metrics(y_test, xgb_test_pred, X_test_enc)
if 'rf_test_metrics' not in globals() and 'rf_test_pred' in globals():
    rf_test_metrics = regression_metrics(y_test, rf_test_pred, X_test_enc)

# Model families to probe (prefix aligns with variable naming pattern)
MODEL_PREFIXES = [
    ('XGBoost', 'xgb'),
    ('RandomForest', 'rf'),
    ('LightGBM', 'lgb'),
    ('SVR', 'svr'),
    ('PolynomialRegression', 'poly'),
    ('LinearRegression', 'lr'),
    ('TorchMLP', 'mlp'),
    ('TabNet', 'tabnet'),
    # Baseline variants (if both baseline and tuned exist you'll see two rows)
    ('TorchMLP_Baseline', 'mlp_baseline'),
    ('TabNet_Baseline', 'tabnet_baseline'),
]

candidates = []
for display_name, prefix in MODEL_PREFIXES:
    valid_pred_var = f'{prefix}_valid_pred'
    test_pred_var = f'{prefix}_test_pred'
    if valid_pred_var in globals():
        v_metrics, t_metrics = _ensure_metrics(prefix, display_name, X_valid_enc, X_test_enc)
        if v_metrics is not None:
            candidates.append({'Model': display_name, 'Valid': deepcopy(v_metrics), 'Test': deepcopy(t_metrics) if t_metrics else None})

# (Legacy metrics variables that don't match prefix pattern already handled above)
if not candidates:
    print('No model metrics available yet. Run earlier training cells first.')
else:
    # Build a long-form DataFrame
    rows = []
    for c in candidates:
        for ds in ['Valid','Test']:
            if c[ds] is None:
                continue
            row = {'Model': c['Model'], 'Dataset': ds}
            row.update(c[ds])
            rows.append(row)
    all_results_df = pd.DataFrame(rows).drop_duplicates(subset=['Model','Dataset']).set_index(['Model','Dataset']).sort_index()

    display(all_results_df)

    # Ranking based on Validation metrics only (composite)
    val_df = all_results_df.xs('Valid', level='Dataset').copy()
    metric_cols = [m for m in ['RMSE','MAE','R2','Adj_R2'] if m in val_df.columns]
    if not metric_cols:
        print('No numeric metrics found for ranking.')
    else:
        ranks = {}
        for m in metric_cols:
            ascending = m in ['RMSE','MAE']  # lower better
            ranks[m + '_rank'] = val_df[m].rank(ascending=ascending, method='min')
        rank_df = pd.DataFrame(ranks)
        val_ranked = val_df.join(rank_df)
        rank_cols = [c for c in val_ranked.columns if c.endswith('_rank')]
        val_ranked['avg_rank'] = val_ranked[rank_cols].mean(axis=1)
        best_model_name = val_ranked['avg_rank'].idxmin()
        best_model_avg_rank = val_ranked.loc[best_model_name, 'avg_rank']
        best_model_metrics_valid = val_df.loc[best_model_name]
        try:
            best_test_metrics = all_results_df.loc[(best_model_name, 'Test')].to_dict()
        except KeyError:
            best_test_metrics = None
        print('\n=== Validation Ranking (lower avg_rank better) ===')
        display(val_ranked.sort_values('avg_rank'))

        print(f"\nOverall Best Model (Validation composite rank): {best_model_name} | avg_rank={best_model_avg_rank:.2f}")
        print('\nBest Model Validation Metrics:')
        for k,v in best_model_metrics_valid.items():
            if not k.endswith('_rank') and k != 'avg_rank':
                print(f'  {k}: {v:.4f}' if isinstance(v,(int,float)) else f'  {k}: {v}')
        if best_test_metrics:
            print('\nBest Model Test Metrics:')
            for k,v in best_test_metrics.items():
                print(f'  {k}: {v:.4f}' if isinstance(v,(int,float)) else f'  {k}: {v}')

        # Metric-wise winners
        metric_winners = {}
        for m in metric_cols:
            ascending = m in ['RMSE','MAE']
            winner = val_df[m].idxmin() if ascending else val_df[m].idxmax()
            metric_winners[m] = {
                'model': winner,
                'value': val_df.loc[winner, m]
            }
        print('\n=== Metric-wise Best (Validation) ===')
        for m, info in metric_winners.items():
            print(f"  {m}: {info['model']} -> {info['value']:.4f}")

        # Persist summary JSON
        summary_path = MODEL_DIR / 'model_comparison_summary.json'
        summary_payload = {
            'models_evaluated': [c['Model'] for c in candidates],
            'validation_table': val_df.to_dict(orient='index'),
            'ranking': val_ranked[['avg_rank']].to_dict(orient='index'),
            'metric_winners': {m: {'model': v['model'], 'value': float(v['value'])} for m,v in metric_winners.items()},
            'overall_best_model': {
                'name': best_model_name,
                'validation_metrics': {k: float(v) for k,v in best_model_metrics_valid.items() if isinstance(v,(int,float))},
                'test_metrics': {k: float(v) if isinstance(v,(int,float)) else v for k,v in (best_test_metrics or {}).items()},
                'avg_rank': float(best_model_avg_rank)
            }
        }
        try:
            with open(summary_path, 'w') as f:
                _json_summary.dump(summary_payload, f, indent=2)
            print(f"\nSaved model comparison summary -> {summary_path.name}")
        except Exception as e:
            print(f"Could not save summary JSON: {e}")

### 6.1 Learning Curves for Ensemble Models

In [ ]:
# Sensitivity Analysis (Partial Dependence style) & Trade-offs Summary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

if best_model_obj is None or 'fi_df' not in globals():
    print('Prerequisites missing: ensure best model & feature importance computed.')
else:
    # Choose top N features for sensitivity sweeps
    TOP_N_SENS = 5
    top_sens_features = fi_df.head(TOP_N_SENS)['feature'].tolist()
    X_valid_dense = X_valid_enc.toarray() if hasattr(X_valid_enc, 'toarray') else np.asarray(X_valid_enc)
    base_preds = best_model_obj.predict(X_valid_dense)
    base_rmse = mean_squared_error(y_valid, base_preds, squared=False)
    sens_records = []

    for feat in top_sens_features:
        if feat not in feature_names:
            continue
        col_idx = feature_names.index(feat)
        col_values = X_valid_dense[:, col_idx]
        # Use quantile grid for numeric-like features. For one-hot (0/1), this will just be [0,1].
        unique_vals = np.unique(col_values)
        if len(unique_vals) <= 2:
            grid = unique_vals
        else:
            grid = np.quantile(col_values, [0.05,0.25,0.5,0.75,0.95])
        preds_at = []
        for gv in grid:
            X_mod = X_valid_dense.copy()
            X_mod[:, col_idx] = gv
            preds_mod = best_model_obj.predict(X_mod)
            rmse_mod = mean_squared_error(y_valid, preds_mod, squared=False)
            preds_at.append({'feature': feat, 'value': float(gv), 'rmse': rmse_mod, 'delta_rmse': rmse_mod - base_rmse})
        sens_records.extend(preds_at)

    sens_df = pd.DataFrame(sens_records)
    display(sens_df.head(20))

    # Plot each feature's sensitivity curve
    n_feat = len(top_sens_features)
    fig, axes = plt.subplots(n_feat, 1, figsize=(7, 3*n_feat), sharex=False)
    if n_feat == 1:
        axes = [axes]
    for ax, feat in zip(axes, top_sens_features):
        sub = sens_df[sens_df['feature'] == feat].sort_values('value')
        ax.plot(sub['value'], sub['rmse'], marker='o')
        ax.axhline(base_rmse, color='gray', linestyle='--', label='Base RMSE')
        ax.set_title(f'Sensitivity: {feat}')
        ax.set_xlabel('Value')
        ax.set_ylabel('RMSE')
        ax.legend()
        ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Trade-offs summary: quantify importance vs. sensitivity (range of RMSE)
    trade_rows = []
    for feat in top_sens_features:
        sub = sens_df[sens_df['feature'] == feat]
        if sub.empty:
            continue
        rmse_range = sub['rmse'].max() - sub['rmse'].min()
        # Importance lookup
        imp = fi_df.loc[fi_df['feature'] == feat, 'importance']
        imp_val = float(imp.iloc[0]) if not imp.empty else np.nan
        trade_rows.append({'feature': feat, 'importance': imp_val, 'rmse_range': rmse_range, 'sensitivity_score': rmse_range / (imp_val + 1e-9)})
    trade_df = pd.DataFrame(trade_rows).sort_values('sensitivity_score', ascending=False)
    print('\nTrade-offs (higher sensitivity_score => large RMSE movement per unit importance):')
    display(trade_df)

    # Persist analysis artifacts
    analysis_payload = {
        'feature_importances': fi_df.to_dict(orient='records') if 'fi_df' in globals() else None,
        'ablation': ablation_df.to_dict(orient='records') if 'ablation_df' in globals() else None,
        'sensitivity': sens_df.to_dict(orient='records'),
        'tradeoffs': trade_df.to_dict(orient='records'),
        'best_model': best_model_name_runtime,
        'base_validation_rmse': float(base_rmse)
    }
    analysis_path = MODEL_DIR / 'best_model_explainability.json'
    try:
        with open(analysis_path, 'w') as f:
            json.dump(analysis_payload, f, indent=2)
        print(f'Saved explainability analysis -> {analysis_path.name}')
    except Exception as e:
        print('Failed to save explainability JSON:', e)

## 6.2 Best Model Deep Dive: Feature Importance, Ablation, Sensitivity & Trade‑offs
This section automatically targets the currently best performing model (based on composite validation rank from the previous cell) and provides:
1. Unified feature name extraction from the preprocessing pipeline.
2. Multiple feature importance strategies (native, permutation).
3. Ablation study: progressive removal (or neutralization) of top features to measure performance degradation.
4. Sensitivity analysis: perturbation / partial dependence style sweeps for top features.
5. Trade‑off discussion metrics (complexity vs. performance deltas).
Run the cells below after the comparison cell has executed.

In [ ]:
# Ablation Analysis: neutralize/remove top-K features and measure RMSE delta
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

if best_model_obj is None or 'fi_df' not in globals():
    print('Prerequisites missing: ensure best model & feature importance computed.')
else:
    base_preds = best_model_obj.predict(X_valid_enc)
    try:
        base_rmse = mean_squared_error(y_valid, base_preds, squared=False)
    except Exception:
        base_rmse = np.sqrt(mean_squared_error(y_valid, base_preds))
    print(f'Base validation RMSE: {base_rmse:.4f}')

    top_features = fi_df.head(30)['feature'].tolist()  # consider top 30 for ablation scope
    ablation_results = []
    # Create a mapping feature_name -> column index for efficient masking
    feat_index_map = {f: i for i, f in enumerate(feature_names)}

    X_valid_dense = X_valid_enc.toarray() if hasattr(X_valid_enc, 'toarray') else np.asarray(X_valid_enc)
    for k in [1,2,3,5,10,15,20,25,30]:
        subset = top_features[:k]
        cols = [feat_index_map[f] for f in subset if f in feat_index_map]
        if not cols:
            continue
        X_mod = X_valid_dense.copy()
        # Neutralization strategy: set to column mean (could also zero)
        for c in cols:
            col_mean = X_mod[:, c].mean()
            X_mod[:, c] = col_mean
        preds_mod = best_model_obj.predict(X_mod)
        rmse_mod = mean_squared_error(y_valid, preds_mod, squared=False)
        ablation_results.append({'k_removed': k, 'rmse': rmse_mod, 'delta_rmse': rmse_mod - base_rmse})
    ablation_df = pd.DataFrame(ablation_results)
    display(ablation_df)

    plt.figure(figsize=(7,4))
    plt.plot(ablation_df['k_removed'], ablation_df['delta_rmse'], marker='o')
    plt.axhline(0, color='gray', linestyle='--')
    plt.xlabel('Number of Top Features Neutralized')
    plt.ylabel('RMSE Delta vs Base')
    plt.title(f'Ablation Impact - {best_model_name_runtime}')
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
# Feature Importance (native if available, else permutation)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

if best_model_obj is None:
    print('Best model object not available; run previous cells.')
else:
    # Try native importances
    importances = None
    importance_type = None
    try:
        if hasattr(best_model_obj, 'feature_importances_'):
            importances = getattr(best_model_obj, 'feature_importances_')
            importance_type = 'feature_importances_'
        elif hasattr(best_model_obj, 'coef_'):
            coef = getattr(best_model_obj, 'coef_')
            # Flatten if necessary
            importances = np.abs(coef.ravel())
            importance_type = 'coef_'
    except Exception as e:
        print('[Importance] Native extraction failed:', e)

    if importances is None or len(importances) != len(feature_names):
        print('[Importance] Falling back to permutation importance (n_repeats=10).')
        try:
            perm = permutation_importance(best_model_obj, X_valid_enc, y_valid, n_repeats=10, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=42)
            importances = np.maximum(0, perm.importances_mean)
            importance_type = 'permutation_neg_rmse'
        except Exception as e:
            print('[Importance] Permutation importance failed:', e)
            importances = None

    if importances is not None:
        fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
        fi_df = fi_df.sort_values('importance', ascending=False).reset_index(drop=True)
        display(fi_df.head(25))
        # Plot
        top_plot = fi_df.head(25).iloc[::-1]
        plt.figure(figsize=(8, max(4, 0.3*len(top_plot))))
        plt.barh(top_plot['feature'], top_plot['importance'], color='steelblue')
        plt.title(f'Top Feature Importances ({importance_type}) - {best_model_name_runtime}')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
    else:
        print('Could not compute feature importance.')

In [ ]:
# Select best model object + extract feature names
from sklearn.inspection import permutation_importance
best_model_name_runtime = None
best_model_obj = None

# If previous cell stored summary JSON, load it to get best model name
summary_path = MODEL_DIR / 'model_comparison_summary.json'
if summary_path.exists():
    try:
        with open(summary_path) as f:
            _summary_json = json.load(f)
        best_model_name_runtime = _summary_json.get('overall_best_model', {}).get('name')
        print(f"[BestModel] Loaded best model from summary JSON: {best_model_name_runtime}")
    except Exception as e:
        print(f"[BestModel] Could not read summary JSON: {e}")
else:
    print('[BestModel] Summary JSON not found. Run comparison cell first.')

# Map display name to the underlying variable (heuristic)
_MODEL_VAR_MAP = {
    'XGBoost': 'xgb_model',
    'RandomForest': 'rf_model',
    'LightGBM': 'lgb_model',
    'CatBoost': 'cat_model',
    'SVR': 'svr_model',
    'PolynomialRegression': 'poly_model',
    'LinearRegression': 'lr_model',
    'TorchMLP': 'mlp_model',
    'TabNet': 'tabnet_model',
}

if best_model_name_runtime and best_model_name_runtime in _MODEL_VAR_MAP:
    var_name = _MODEL_VAR_MAP[best_model_name_runtime]
    best_model_obj = globals().get(var_name)
    if best_model_obj is None:
        print(f"[BestModel] Variable '{var_name}' not found or is None; ensure model cell was executed.")
else:
    print('[BestModel] Could not determine model variable mapping.')

# Extract feature names after preprocessing
def get_feature_names(preprocessor):
    feat_names = []
    try:
        # Numeric
        for name, trans, cols in preprocessor.transformers_:
            if name == 'remainder':
                continue
            if name == 'num':
                feat_names.extend(cols)
            elif name == 'cat':
                # OneHotEncoder inside pipeline
                try:
                    ohe = trans.named_steps['onehot'] if hasattr(trans, 'named_steps') else trans
                    cats = ohe.get_feature_names_out(cols)
                    feat_names.extend(cats)
                except Exception as e:
                    print('[FeatureNames] OHE extraction failed:', e)
    except Exception as e:
        print('[FeatureNames] Fallback feature name generation:', e)
        # Fallback to generic idx names
        feat_names = [f'f{i}' for i in range(X_train_enc.shape[1])]
    if len(feat_names) != X_train_enc.shape[1]:
        # Align length via fallback if mismatch
        print('[FeatureNames] Length mismatch; falling back to positional feature names.')
        feat_names = [f'f{i}' for i in range(X_train_enc.shape[1])]
    return feat_names

feature_names = get_feature_names(preprocessor)
print(f"Extracted {len(feature_names)} feature names.")
if best_model_name_runtime:
    print(f"Proceeding with best model: {best_model_name_runtime}")
else:
    print('Best model not established yet. Subsequent cells may fail.')

In [ ]:
# Learning curves for ensemble models
from sklearn.model_selection import learning_curve
from sklearn.base import clone

def plot_learning_curve(estimator, X, y, ax, title, cv=5, scoring='neg_root_mean_squared_error'):
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator,
        X,
        y,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 6),
        shuffle=True,
        random_state=42
    )
    train_rmse = -train_scores.mean(axis=1)
    valid_rmse = -valid_scores.mean(axis=1)

    ax.plot(train_sizes, train_rmse, label='Train RMSE', marker='o')
    ax.fill_between(train_sizes, train_rmse - train_scores.std(axis=1), train_rmse + train_scores.std(axis=1), alpha=0.2)
    ax.plot(train_sizes, valid_rmse, label='Validation RMSE', marker='s')
    ax.fill_between(train_sizes, valid_rmse - valid_scores.std(axis=1), valid_rmse + valid_scores.std(axis=1), alpha=0.2)
    ax.set_title(title)
    ax.set_xlabel('Training Examples')
    ax.set_ylabel('RMSE')
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.4)

xgb_curve_model = clone(xgb_model)
rf_curve_model = clone(rf_model)


fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
plot_learning_curve(xgb_curve_model, X_train_enc, y_train, axes[0], 'XGBoost Learning Curve')
plot_learning_curve(rf_curve_model, X_train_enc, y_train, axes[1], 'Random Forest Learning Curve')
plt.tight_layout()
plt.show()